In [135]:
import plotly
import plotly.graph_objects as go
import numpy as np

Our object will be a wireframe cube.

In [143]:
def get_cube(R: np.ndarray = None, t: np.ndarray = None) -> tuple[list, list]:
    # Get base vertices: unit cube centered at the origin
    vertices = [
        (-0.5,-0.5,-0.5),
        (0.5,-0.5,-0.5),
        (0.5,0.5,-0.5),
        (-0.5,0.5,-0.5),
        (-0.5,-0.5,0.5),
        (0.5,-0.5,0.5),
        (0.5,0.5,0.5),
        (-0.5,0.5,0.5)
    ]
    # Get connectivity (this doesn't change with transformations of the cube)
    edges = [
        (0,1), (1,2), (2,3), (3,0),     # front face
        (4,5), (5,6), (6,7), (7,4),     # back face
        (0,4), (1,5), (2,6), (3,7),     # connecting edges
    ]
    # Apply a rotation
    if R is not None:
        for idx, v in enumerate(vertices):
            r = R @ np.array(v)
            vertices[idx] = (float(r[0]), float(r[1]), float(r[2]))
    # Apply a translation
    if t is not None:
        for idx, v in enumerate(vertices):
            vertices[idx] = (t[0] + v[0], t[1] + v[1], t[2] + v[2])
    
    return vertices, edges

def plot_cube(vertices: list, edges: list, fig: go.Figure):
    v_x = [v[0] for v in vertices]
    v_y = [v[1] for v in vertices]
    v_z = [v[2] for v in vertices]
    
    fig.add_trace(
        go.Scatter3d(
            x=v_x,
            y=v_y,
            z=v_z,
            mode='markers',
            marker=dict(color='red', size=5),\
            showlegend=False,
        )
    )
    for (v1_idx, v2_idx) in edges:
        v1, v2 = vertices[v1_idx], vertices[v2_idx]
        fig.add_trace(
            go.Scatter3d(
                x=[v1[0], v2[0]],
                y=[v1[1], v2[1]],
                z=[v1[2], v2[2]],
                mode='lines',
                line=dict(color='black'),
                showlegend=False,
            )
        )

def _get_image_plane():
    vertices = [
        (-1, -1, 1),
        (1, -1, 1),
        (1, 1, 1),
        (-1, 1, 1)
    ]
    edges = [(0,1), (1,2), (2,3), (3,0)]
    return vertices, edges

def draw_image_plane(fig: go.Figure):
    vertices, edges = _get_image_plane()
    for (v1_idx, v2_idx) in edges:
        v1, v2 = vertices[v1_idx], vertices[v2_idx]
        fig.add_trace(
            go.Scatter3d(
                x=[v1[0], v2[0]],
                y=[v1[1], v2[1]],
                z=[v1[2], v2[2]],
                mode='lines',
                line=dict(color='black', width=0.5),
                showlegend=False,
            )
        )



In [144]:
theta = np.deg2rad(45)
R_y = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])

vertices, edges = get_cube(R=R_y, t=[0,0,3])

In [145]:
fig = go.Figure()
plot_cube(vertices=vertices, edges=edges, fig=fig)

draw_image_plane(fig)

# Plot camera center
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], mode='markers', marker=dict(size=5, color='black'), showlegend=False))
fig.update_layout(
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1), # equal scaling for all axes
        xaxis=dict(range=[-5,5]),
        yaxis=dict(range=[-5,5]),
        zaxis=dict(range=[-2,8]),
        camera=dict(
            eye=dict(x=1,y=-1,z=-1),
            center=dict(x=0,y=0,z=0),
            up=dict(x=0,y=-1,z=0)
        )
    )
)

fig.show()

In [146]:
# Extrinsics are trivial in this case
R = np.eye(3)
t = np.zeros(3)

def do_projection_onto_norm_img_plane(
        points: list,
        extrinsics: tuple[np.ndarray, np.ndarray]
    ) -> np.ndarray:

    R, t = extrinsics

    norm_img_pts = []
    for v in points:
        # Object point in world coordinates
        x_w = np.array(v)
        # Map to camera coordinates (trivial here)
        x_c = R @ x_w + t
        # Project onto normalized image plane
        x_nimg = x_c / x_c[2]

        norm_img_pts.append(x_nimg)

    norm_img_pts = np.stack(norm_img_pts)
    return norm_img_pts

In [147]:
fig = go.Figure()

# Extrinsics are trivial in this case
R = np.eye(3)
t = np.zeros(3)

# Perform projection
nimp = do_projection_onto_norm_img_plane(vertices, extrinsics=(R,t))

fig.add_trace(go.Scatter(
    x=list(nimp[:,0]),
    y=list(nimp[:,1]),
    mode='markers',
    marker=dict(color='red'),
    showlegend=False,
))
for (v1_idx, v2_idx) in edges:
    v1, v2 = nimp[v1_idx, :], nimp[v2_idx, :]
    fig.add_trace(
        go.Scatter(
            x=[v1[0], v2[0]],
            y=[v1[1], v2[1]],
            mode='lines',
            line=dict(color='black', width=0.5),
            showlegend=False,
        )
    )


fig.show()

In [148]:
fig = go.Figure()

# Plot the object points in 3D
plot_cube(vertices=vertices, edges=edges, fig=fig)

# Plot camera center
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], mode='markers', marker=dict(size=5, color='black'), showlegend=False))

# Draw the image plane
draw_image_plane(fig)

# Draw the projection in the image plane
fig.add_trace(go.Scatter3d(
    x=list(nimp[:,0]),
    y=list(nimp[:,1]),
    z=list(nimp[:,2]),
    mode='markers',
    marker=dict(color='red', size=1),
    showlegend=False,
))
for (v1_idx, v2_idx) in edges:
    v1, v2 = nimp[v1_idx, :], nimp[v2_idx, :]
    fig.add_trace(
        go.Scatter3d(
            x=[v1[0], v2[0]],
            y=[v1[1], v2[1]],
            z=[1, 1],
            mode='lines',
            line=dict(color='black', width=0.5),
            showlegend=False,
        )
    )

fig.update_layout(
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1), # equal scaling for all axes
        xaxis=dict(range=[-5,5]),
        yaxis=dict(range=[-5,5]),
        zaxis=dict(range=[-2,8]),
        camera=dict(
            eye=dict(x=0.25,y=-0.25,z=-0.55),
            center=dict(x=0,y=0,z=0),
            up=dict(x=0,y=-1,z=0)
        )
    )
)


fig.show()

In [149]:
fig = go.Figure()

# Plot the object points in 3D
plot_cube(vertices=vertices, edges=edges, fig=fig)

# Plot camera center
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], mode='markers', marker=dict(size=5, color='black'), showlegend=False))

# Draw the image plane
draw_image_plane(fig)

# Draw the projection in the image plane
fig.add_trace(go.Scatter3d(
    x=list(nimp[:,0]),
    y=list(nimp[:,1]),
    z=list(nimp[:,2]),
    mode='markers',
    marker=dict(color='red', size=1),
    showlegend=False,
))
for (v1_idx, v2_idx) in edges:
    v1, v2 = nimp[v1_idx, :], nimp[v2_idx, :]
    fig.add_trace(
        go.Scatter3d(
            x=[v1[0], v2[0]],
            y=[v1[1], v2[1]],
            z=[1, 1],
            mode='lines',
            line=dict(color='black', width=0.5),
            showlegend=False,
        )
    )

fig.update_layout(
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1), # equal scaling for all axes
        xaxis=dict(range=[-5,5]),
        yaxis=dict(range=[-5,5]),
        zaxis=dict(range=[-2,8]),
        camera=dict(
            eye=dict(x=0,y=0,z=-0.5),
            center=dict(x=0,y=0,z=1),
            up=dict(x=0,y=-1,z=0)
        )
    )
)


fig.show()